In [88]:
import numpy as np
from equations import *
from solvers import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
N = 20
J = 20

params = generate_test_parameters(N=N, J=J)
shocks = generate_test_shocks(params)

In [94]:
sol = solve_equilibrium(params, shocks, 0, mute=False)

Round 1: wfmax=0.0000, min_w_hat=1.0000, max_w_hat=1.0000, min_Xf_prime=5.0000, max_Xf_prime=5.0000, min_Xm_prime=11.6667, max_Xm_prime=11.6667
Converged in 1 iterations


In [100]:
alpha = 0.1

# Initialize the variables
N, J = params.N, params.J
VA = np.sum(params.w0 * params.L0)

# Initlial guess for wage rate changes
w_hat = np.ones(N)

# Initial guess for the price index changes of intermediate goods
Pm_hat = np.ones((N, J))

# Initialize trade deficit
td_prime = params.td

wfmax = 1.0


c_hat, Pm_hat = solve_price_and_cost(
    w_hat, Pm_hat, params, shocks, max_iter=1000, tol=1e-6, mute=True
)
Pf_hat = calc_Pu_hat(c_hat, "f", params, shocks)
pif_hat = calc_piu_hat(c_hat, Pf_hat, "f", params, shocks)
pim_hat = calc_piu_hat(c_hat, Pm_hat, "m", params, shocks)
Xf_prime, Xm_prime = solve_X_prime_analytical(w_hat, pif_hat, pim_hat, td_prime, params, shocks)

td_prime = calc_td_prime(
    pif_hat, pim_hat, Xf_prime, Xm_prime, params, shocks
)

# ZW2 captures the difference
# between the calculated trade deficit and the target trade deficit
# normalized by the initial value added

VA_prime = np.sum(params.w0 * params.L0 * w_hat)

ZW2 = td_prime / VA_prime - params.td / VA

# Update the wage changes (numeraire country is excluded)
w_hat_new = np.ones(N)
w_hat_new = w_hat * np.exp(-alpha * ZW2)


# Shock Simulation

## Simulate $\hat{\Lambda}_i^j$

Assume $\hat{\Lambda}_i^j$ is log-normal, i.e.$ \hat{\Lambda}_i^j \sim \text{exp}(\mathcal{N}(0, \sigma))$. For the simplist case, we are assuming the shocks of all country $i$ and sector $j$ are i.i.d. and share the same variance captured by $\sigma$. Below is the corresponding level of $\log \Lambda$ and hence $\Lambda$ for $\sigma \in \{0.1, 0.2, 0.3\}$:

|  $\sigma$  | 95% Interval for $\log  \Lambda$  | 95% Interval for $\Lambda$            |
|:----------:|:---------------------------------:|:-------------------------------------:|
| 0.1        | $[-0.196, 0.196]$             | [ $\exp(-0.196) \approx 0.82,\, \exp(0.196) \approx 1.22]$ |
| 0.2        | $[-0.392,\, 0.392]$             | $[ \exp(-0.392) \approx 0.68,\, \exp(0.392) \approx 1.48]$ |
| 0.3        | $[-0.588,\, 0.588]$             | $[ \exp(-0.588) \approx 0.56,\, \exp(0.588) \approx 1.80]$ |

In [115]:
# pre-specifiy N and J
N = 10
J = 10
sigmas = [0.1, 0.2, 0.3]
num_sim = 100
Lambdas = np.zeros((len(sigmas), num_sim, N, J))
for i, sigma in enumerate(sigmas):
    Lambdas[i] = np.exp(np.random.normal(loc=0.0, scale=sigma, size=(num_sim, N, J)))
